In [1]:
import pandas as pd
from custom_functions_scRBA import *

In [42]:
df_rxns = pd.read_excel('./scRAM_network.xlsx')
df_rxns.index = df_rxns.id.to_list()
df_rxns = df_rxns[df_rxns.symbolic_coeffs.isnull() == False]

df_keff = pd.read_excel('./keff.xlsx')
df_keff.index = df_keff.rxn_id.to_list()

df_pro = pd.read_excel('../scRBA/curation/prot_stoich_curation_2021-05-21.xlsx')
df_pro.index = df_pro.id.to_list()

df_params = pd.read_excel('../scRBA/input/parameters_info.xlsx')
df_params.index = df_params.param_id.to_list()

In [40]:
kribo = 10.5; dummyNAA = 405;

#### Build symbolic stoichiometry

In [43]:
stoich = ['/', '* List of stoichiometric relations']
for i in df_rxns.index:
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[i])
    met_dict = {k:v for k,v in met_dict.items() if k != '' and is_number(v) == False}
    sym = df_rxns.symbolic_coeffs[i]
    
    for k,v in met_dict.items():
        if v == '-mu/keff':
            v = str(round(-1 / df_keff.keff[i], 12)) + '*mu'
        elif v == '-mu*NAA/kribo':
            pro_id = i[7:]
            if pro_id in ['DUMMYPRO_c', 'DUMMYPRO_m']:
                v = str(round(dummyNAA / kribo, 12)) + '*mu'
            else:
                v = str(round((len(df_pro.sequence[pro_id]) - 1) / kribo, 12)) + '*mu'
        else:
            v = df_params.string[v]
            
        stoich.append("'" + i + "'.'" + k + "' " + str(v))
            
stoich.append('/')
with open('./GAMSmodel/sij_symbolic.txt', 'w') as f:
    f.write('\n'.join(stoich))

#### Merge two lists of stoichiometry: constant and symbolic

In [56]:
with open('./GAMSmodel/sij_constant.txt') as f:
    sij_constant = f.read().split('\n')
sij_constant = [i for i in sij_constant if i not in ['/', ''] and i [0] != '*']

with open('./GAMSmodel/sij_symbolic.txt') as f:
    sij_sym = f.read().split('\n')
sij_sym = [i for i in sij_sym if i not in ['/', ''] and i [0] != '*']

In [66]:
sij = sij_constant + sij_sym
sij = ['/'] + sij + ['/']
with open('./GAMSmodel/sij_merged_lower_0_211.txt', 'w') as f:
    f.write('\n'.join(sij))